# 1. Networked Data and Model

This notebook contains the workflow for the first milestone in the Manning liveProject *Personalized Diagnosis of Symptoms*. *Personalized Diagnosis of Symptoms* is part of the series *Federated Learning Over Networks for Pandemics*.

In [1]:
import numpy as np
import networkx as nx
import torch

In [2]:
B = 30 # size of the blocks
n=1# number of weights = number of features
m=5 # sample size

## Generate network

In [3]:
sizes = [B] * 3
probs = [[0.8, 0.1, 0.1], [0.1, 0.8, 0.1], [0.1, 0.1, 0.8]]

G = nx.stochastic_block_model(sizes, probs, seed=0)

# set weights for each block - am I doing this right?
b1_weights = torch.zeros((1,n))
b2_weights = torch.zeros((1,n))
b3_weights = torch.zeros((1,n))

## Set node attributes

In [4]:
for i in G.nodes:
    G.nodes[i]["weight"] = np.zeros((1,n))
    G.nodes[i]["cluster"] = G.nodes[i]["block"] + 1
    G.nodes[i]["features"] =torch.randn(size=(n,m))
    
    # labels are the products of features and block weights
    if G.nodes[i]["block"]==0:
        G.nodes[i]["label"] = torch.matmul(b1_weights,G.nodes[i]["features"])
    if G.nodes[i]["block"]==1:
        G.nodes[i]["label"] = torch.matmul(b2_weights,G.nodes[i]["features"])
    if G.nodes[i]["block"]==2:
        G.nodes[i]["label"] = torch.matmul(b3_weights,G.nodes[i]["features"])
    
    G.nodes[i]["model"] = torch.nn.Linear(in_features=n, out_features=1, bias=False)
    G.nodes[i]["optimizer"] = torch.optim.LBFGS(G.nodes[i]["model"].parameters())
    G.nodes[i]["criterion"] = torch.nn.MSELoss(size_average=True)

/Users/t-r-espr/opt/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


## Set edge attributes

In [5]:
for i,j in G.edges:
    G.edges[i,j]["weight"] = np.zeros((1,n))